In [1]:
%matplotlib inline
import subprocess
import netCDF4
import numpy as np
from osgeo import gdal
from osgeo.gdalconst import *
import pytz
import itertools

Parse http://opendap.tudelft.nl/thredds/data2/zandmotor/morphology/LIDAR/catalog.html ...

In [31]:
# A bit ugly, with requests and beautiful soup, but it collects the files from on TUDelft repository

from bs4 import BeautifulSoup
import requests

urlpath = 'http://opendap.tudelft.nl/thredds/data2/zandmotor/morphology/LIDAR/catalog.html'
ext = 'nc'
urls = []

def listFD(url, ext=''):
    page = requests.get(urlpath).text
    #print(page)
    soup = BeautifulSoup(page, 'html.parser')
    return [urlpath + '/' + node.get('href') for node in soup.find_all('a') if node.get('href').endswith(ext)]

for ncfile in listFD(url, ext):
    nclist = ncfile.partition('/catalog.html/')[0] + ncfile[-16:]
    urls.append(nclist.replace('thredds/data2','thredds/dodsC/data2'))

print(urls[0])

http://opendap.tudelft.nl/thredds/dodsC/data2/zandmotor/morphology/LIDAR/72hz2_res5x5.nc


In [39]:
from tqdm import tqdm

grids = []
for url in tqdm(urls):
    #print(url)
    ds = netCDF4.Dataset(url)
    times = netCDF4.num2date(ds.variables['time'][:], ds.variables['time'].units)
    local = pytz.timezone("Europe/Amsterdam")
    times = [local.localize(t, is_dst=None).astimezone(pytz.utc) for t in times]
    arrs = []
    z = ds.variables['z'][:]
    x = ds.variables['x'][:]
    y = ds.variables['y'][:]
    grids.append({
        "url": url,
        "x": x,
        "y": y,
        "z": z,
        "times": times
    })
    ds.close()


100%|██████████| 106/106 [01:01<00:00,  1.71it/s]


In [38]:
count = len(list(itertools.chain.from_iterable([g['times'] for g in grids])))
count

111

In [6]:
ee_collection_path = 'users/gena/eo-bathymetry/sandengine_lidar'

In [11]:
def run(cmd):
    print(cmd)
    subprocess.call(cmd)

In [ ]:
i = 0
for g in grids:
    ncols = len(g['x'])
    nrows = len(g['y'])
    cellsize = g['x'][1] - g['x'][0]
    xllcorner = np.min(g['x'])
    yllcorner = np.min(g['y'])
    nodata_value = -9999

    for t in g['times']:
        print('Processing time: {0}'.format(t))
        z = g['z']
        filename = 'bathymetry_lidar_' + str(i).rjust(3, '0')
        filepath = '../output/' + filename
        filepath_asc = filepath + '.asc'
        filepath_tif = filepath + '.tif'
        filename_tif = filename + '.tif'

        z = z[i]

        with open(filepath_asc, 'w') as f:
            f.write('ncols {0}\n'.format(ncols))
            f.write('nrows {0}\n'.format(nrows))
            f.write('cellsize {0}\n'.format(cellsize))
            f.write('xllcorner {0}\n'.format(xllcorner))
            f.write('yllcorner {0}\n'.format(yllcorner))
            f.write('nodata_value {0}\n'.format(nodata_value))
            for row in np.flip(range(nrows), 0):
                s = ' '.join([str(v) for v in z[row,]]).replace('--', str(nodata_value))
                f.write(s)
                f.write('\n')

        cmd = 'gdal_translate -ot Float32 -a_srs EPSG:28992 -co COMPRESS=DEFLATE -co PREDICTOR=2 -co ZLEVEL=6 -of GTiff {0} {1}'\
            .format(filepath_asc, filepath_tif)
        run(cmd)
        
        filepath_gs = 'gs://hydro-earth/sandengine/' + filename_tif
        
        gsutil = 'D:/src/google-cloud-sdk/bin/gsutil.cmd' # relative path is not defined on Windows
        cmd = gsutil + ' cp {0} {1}'\
            .format(filepath_tif, filepath_gs)
        run(cmd)
        
        filepath_ee = ee_collection_path + '/' + filename        
        cmd = 'earthengine upload image --wait --asset_id={0} --nodata_value={1} {2}'\
            .format(filepath_ee, nodata_value, filepath_gs)        
        run(cmd)
        
        time_start = int(grids[0]['times'][0].timestamp() * 1000)
        cmd = 'earthengine asset set --time_start {0} {1}'\
            .format(time_start, filepath_ee)
        run(cmd)

        cmd = 'earthengine acl set public {0}'\
            .format(filepath_ee)
        run(cmd)

        i = i + 1

Processing time: 2011-08-02 22:00:00+00:00
